In [27]:
from importlib import reload
import xarray as xr
import dominosee.es as es

In [29]:
import dask.config
dask.config.set(scheduler='processes')

# Calculate ES on unstacked and stacked data

In [34]:
ds = xr.open_dataset("tests/data/era5.reanalysis.spi30d.0p25deg.china.1950-1979.nc").rename({"latitude": "lat", "longitude": "lon", "spi30d": "SPI1"})\
        .isel(lat=slice(0, 25), lon=slice(0, 20), time=slice(0, 2000)).load()
ds_stack = ds.stack(location=["lat", "lon"])
ds

<xarray.Dataset> Size: 4MB
Dimensions:  (time: 2000, lon: 20, lat: 25)
Coordinates:
  * time     (time) datetime64[ns] 16kB 1950-01-01 1950-01-02 ... 1955-06-23
  * lon      (lon) float64 160B 73.0 73.25 73.5 73.75 ... 77.0 77.25 77.5 77.75
  * lat      (lat) float64 200B 54.25 54.0 53.75 53.5 ... 49.0 48.75 48.5 48.25
Data variables:
    SPI1     (time, lat, lon) float32 4MB -0.531 -0.5337 ... -1.504 -1.505
Attributes:
    CDI:          Climate Data Interface version 2.4.4 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Thu Mar 06 20:30:32 2025: cdo -L selindexbox,293,542,144,34...
    CDO:          Climate Data Operators version 2.4.4 (https://mpimet.mpg.de...

In [35]:
reload(es)
event_ds = es.get_event_positions(ds.SPI1 < -1.5)
event_ds_stack = es.get_event_positions(ds_stack.SPI1 < -1.5)
event_ds

<xarray.Dataset> Size: 753kB
Dimensions:          (lon: 20, lat: 25, event: 373)
Coordinates:
  * lon              (lon) float64 160B 73.0 73.25 73.5 ... 77.25 77.5 77.75
  * lat              (lat) float64 200B 54.25 54.0 53.75 ... 48.75 48.5 48.25
  * event            (event) int64 3kB 0 1 2 3 4 5 6 ... 367 368 369 370 371 372
Data variables:
    event_positions  (lat, lon, event) int32 746kB 37 38 39 40 ... -1 -1 -1 -1
    event_count      (lat, lon) int64 4kB 267 267 270 267 ... 175 162 116 115

In [36]:
event_diff_da = es.get_event_time_differences(event_ds.event_positions, event_ds.event_count)
event_diff_da_stack = es.get_event_time_differences(event_ds_stack.event_positions, event_ds_stack.event_count)
event_diff_da

<xarray.DataArray 'event_positions' (lat: 25, lon: 20, event: 373)> Size: 1MB
array([[[ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        ...,
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan]],

       [[ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        ...,
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [18., 32.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan]],

       [[ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        ...,
...
        ...,
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan]],

       [[ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        ...,
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan]],

       [[ 1.,  1., 95., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        ...,
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan],
        [ 1.,  1.,  1., ..., nan, nan, nan]]], shape=(25, 20, 373))
Coordinates:
  * lon      (lon) float64 160B 73.0 73.25 73.5 73.75 ... 77.0 77.25 77.5 77.75
  * lat      (lat) float64 200B 54.25 54.0 53.75 53.5 ... 49.0 48.75 48.5 48.25
  * event    (event) int64 3kB 0 1 2 3 4 5 6 7 ... 366 367 368 369 370 371 372
Attributes:
    long_name:    Event Time Differences
    units:        time steps
    description:  Time differences between consecutive events for each locati...

In [ ]:
reload(es)
es_ds = es.get_event_sync_from_positions(event_ds.event_positions, event_ds.event_positions.copy(), 
                                 event_diff_da, event_diff_da.copy(),
                                 event_ds.event_count, event_ds.event_count.copy(),
                                 )
es_ds_stack = es.get_event_sync_from_positions(event_ds_stack.event_positions, event_ds_stack.event_positions.copy(), 
                                 event_diff_da_stack, event_diff_da_stack.copy(),
                                 event_ds_stack.event_count, event_ds_stack.event_count.copy(),
                                 )
es_ds_stack

<xarray.DataArray (locationA: 500, locationB: 500)> Size: 500kB
array([[265, 258, 249, ...,  30,  29,  37],
       [258, 265, 256, ...,  28,  27,  35],
       [249, 256, 268, ...,  27,  26,  34],
       ...,
       [ 29,  27,  26, ..., 160, 107,  93],
       [ 29,  27,  26, ..., 108, 114,  92],
       [ 37,  35,  34, ...,  94,  92, 113]],
      shape=(500, 500), dtype=uint16)
Coordinates:
  * locationA  (locationA) object 4kB MultiIndex
  * latA       (locationA) float64 4kB 54.25 54.25 54.25 ... 48.25 48.25 48.25
  * lonA       (locationA) float64 4kB 73.0 73.25 73.5 ... 77.25 77.5 77.75
  * locationB  (locationB) object 4kB MultiIndex
  * latB       (locationB) float64 4kB 54.25 54.25 54.25 ... 48.25 48.25 48.25
  * lonB       (locationB) float64 4kB 73.0 73.25 73.5 ... 77.25 77.5 77.75
Attributes:
    long_name:     Event Synchronization
    units:         count
    description:   Number of synchronized events between locations A and B
    threshold:     inf
    output_dtype:  <class 'numpy.uint16'>

In [44]:
assert es_ds.stack(locationA=["latA", "lonA"], locationB=["latB", "lonB"]).equals(es_ds_stack)